In [ ]:
import pandas as pd
from pyprojroot import here

from nutils import get_history

import os

# Set option to display all columns
# pd.set_option('display.max_columns', None)

In [ ]:
ORIGIN = 8

In [ ]:
data = pd.read_csv(here() / 'data/interim/data.csv', parse_dates=True, index_col='Datetime')

In [ ]:
# Targets
mask = data.columns.str.startswith('Target-Q90')
targets = data.loc[:,mask]

targets =  data.loc[:,mask].groupby([data.index.date, data.index.hour]).first().unstack().T.groupby(level=0).sum().T
targets = (targets>=3).astype(int)

In [ ]:
# Histories
mask = data.columns.str.startswith('Target-Q90')
histories = data.loc[:,mask]

a = list()

for c in histories.columns:
    v = histories.loc[:,c]
    m = get_history(v, 24*7, v.name)
    a.append(m)

histories = pd.concat(a, axis=1)
histories = histories.iloc[ORIGIN::24, : ]
histories.index = histories.index.date

In [ ]:
# Calendar
mask = data.columns.str.startswith('Calendar')
calendar = data.loc[:,mask]
calendar = calendar.resample('D').first()
calendar.index = calendar.index.date

In [ ]:
# Edor
mask = data.columns.str.startswith('Target-Edor')
edor = data.loc[:,mask]

a = list()

for c in edor.columns:
    v = edor.loc[:,c]
    m = get_history(v, 24*7, v.name)
    a.append(m)

edor = pd.concat(a, axis=1)
edor = edor.iloc[ORIGIN::24, : ]
edor.index = edor.index.date

#edor = edor.groupby([edor.index.date, edor.index.hour]).first()
#idx = pd.IndexSlice
#edor = edor.loc[idx[:, :ORIGIN], :].unstack()
#edor.columns = [x[0] + '_' + str(x[1]) for x in edor.columns.to_flat_index()]

In [ ]:
# Beds
mask = data.columns.str.startswith('Beds')
beds = data.loc[:,mask]


In [ ]:
beds = beds[beds.index.hour==ORIGIN]
beds.index = beds.index.date

In [ ]:
# Weather
mask = data.columns.str.startswith('Weather')
weather = data.loc[:,mask]
weather = weather.resample('D').first()
weather.index = weather.index.date

In [ ]:
# Crowding
c7 = targets.shift(7)
c7.columns = [f'Crowding_{x}_{7}' for x in c7.columns]

c14 = targets.shift(14)
c14.columns = [f'Crowding_{x}_{14}' for x in c14.columns]

crowding = pd.concat([c7, c14], axis=1)

In [ ]:
data = pd.concat([targets, histories, calendar, edor, beds, crowding, weather], axis=1)

In [ ]:
data.index.name = 'Datetime'

In [ ]:
folder = here() / 'data/model/'

if not os.path.exists(folder):
    os.makedirs(folder)

data.to_csv(folder / f'data-{ORIGIN}.csv')